---
layout: post
title: Sprint5 Blog
description:  Leaderboard
type: issues 
comments: true
---

Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.

Individual
My individual feature is primarily focused on making a leaderboard to showcase your scores. This feature allows people to see their highscores, low scores, and challenge themselves with default scores to earn a better score. The purpose of the feature is to also allow for a more gamelike experience for players.



## Fullstack


# Leaderboard API Technical Analysis

## 1. List Requests, Use of Lists, Dictionaries, and Database




## List Requests  
The `get` method retrieves all leaderboard entries and returns them as a JSON response.  

## Use of Lists and Dictionaries  
The `entries` variable is a list of `LeaderboardEntry` objects, and each entry is converted to a dictionary format for the JSON response.  

## Formatting Response Data  
The JSON response is used to update the DOM in a web application, displaying the leaderboard entries.  

## Database Queries  
SQLAlchemy is used to query the database, returning Python lists representing rows of data.  

## Methods in Class  
The `_LeaderboardCRUD` class defines methods to create, read, update, and delete entries, working with the columns of the database.  


In [ ]:
class _LeaderboardCRUD(Resource):
    def post(self):
        """Create a new leaderboard entry"""
        try:
            data = request.get_json()
            if not data:
                return {'message': 'No data provided'}, 400

            entry = LeaderboardEntry(
                profile_name=data.get('profile_name'),
                drawing_name=data.get('drawing_name'),
                score=int(data.get('score', 0))
            )
            
            if entry.create():
                return {'message': 'Entry added successfully'}, 201
            return {'message': 'Failed to add entry'}, 400

        except Exception as e:
            db.session.rollback()
            return {'error': str(e)}, 500

    def get(self):
        """Retrieve all leaderboard entries"""
        try:
            # query all entries in descending order of score (SQLAlchemy)
            entries = LeaderboardEntry.query.order_by(
                LeaderboardEntry.score.desc()
            ).all()
            return jsonify([entry.read() for entry in entries]), 200
        except Exception as e:
            return {'error': str(e)}, 500

    def put(self):
        """Update a leaderboard entry"""
        try:
            data = request.get_json()
            if not data:
                return {'message': 'No data provided'}, 400

            existing_entry = LeaderboardEntry.query.filter_by(
                profile_name=data.get('profile_name'),
                drawing_name=data.get('drawing_name')
            ).first()

            new_score = int(data.get('score', 0))

            if existing_entry:
                existing_entry.score = new_score
                db.session.commit()
                return {'message': 'Score updated successfully'}, 200

            entry = LeaderboardEntry(
                profile_name=data.get('profile_name'),
                drawing_name=data.get('drawing_name'),
                score=new_score
            )
            
            if entry.create():
                return {'message': 'New entry created'}, 201
            return {'message': 'Failed to create entry'}, 400

        except Exception as e:
            db.session.rollback()
            return {'error': str(e)}, 500

    def delete(self):
        """Delete a leaderboard entry"""
        try:
            data = request.get_json()
            if not data:
                return {'message': 'No data provided'}, 400

            profile_name = data.get('profile_name')
            drawing_name = data.get('drawing_name')

            if not profile_name or not drawing_name:
                return {'message': 'Missing profile_name or drawing_name'}, 400

            entry = LeaderboardEntry.query.filter_by(
                profile_name=profile_name,
                drawing_name=drawing_name
            ).first()

            if not entry:
                return {'message': 'Entry not found'}, 404

            entry.delete()
            return {'message': 'Entry deleted successfully'}, 200

        except Exception as e:
            db.session.rollback()
            return {'error': str(e)}, 500

## 2. Algorithmic code request
The API class and methods are defined to handle different types of HTTP requests (GET, POST, PUT, DELETE). Each method in the class corresponds to a specific CRUD operation.

#### Code showing this:

In [ ]:
@leaderboard_api.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    try:
        entries = LeaderboardEntry.query.order_by( # query all entries in descending order of score (SQLAlchemy)
            LeaderboardEntry.score.desc()
        ).all()
        return jsonify([entry.read() for entry in entries]), 200 
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@leaderboard_api.route('/api/leaderboard', methods=['POST'])
def add_leaderboard_entry():
    try:
        data = request.get_json() # Get data from request
        if not data:
            return jsonify({"error": "No data provided"}), 400

        entry = LeaderboardEntry( # Create new entry
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=int(data.get('score', 0))
        )
        
        if entry.create():
            return jsonify({"message": "Entry added successfully"}), 201
        return jsonify({"error": "Failed to add entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


@leaderboard_api.route('/api/leaderboard', methods=['PUT'])
def update_leaderboard_entry():
    try:
        data = request.get_json() # Get data from request
        if not data:
            return jsonify({"error": "No data provided"}), 400

        # Find existing entry
        existing_entry = LeaderboardEntry.query.filter_by(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name')
        ).first()

        new_score = int(data.get('score', 0))

        # If entry exists, update it regardless of score
        if existing_entry:
            existing_entry.score = new_score  # Always update score
            db.session.commit()
            return jsonify({"message": "Score updated successfully"}), 200

        # If no existing entry, create new one
        entry = LeaderboardEntry(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=new_score
        )
        
        if entry.create():
            return jsonify({"message": "New entry created"}), 201
        return jsonify({"error": "Failed to create entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


@leaderboard_api.route('/api/leaderboard', methods=['DELETE'])
def delete_entry():
    try:
        # Try to get data from JSON body
        data = request.get_json()
        
        # Get profile_name and drawing_name from either JSON body or URL parameters
        profile_name = data.get('profile_name') if data else None
        drawing_name = data.get('drawing_name') if data else None

        # Validate input
        if not profile_name or not drawing_name:
            return jsonify({"error": "Missing profile_name or drawing_name"}), 400

        # Find and delete entry
        entry = LeaderboardEntry.query.filter_by(
            profile_name=profile_name,
            drawing_name=drawing_name
        ).first()

        if not entry:
            return jsonify({"error": "Entry not found"}), 404

        entry.delete()
        return jsonify({"message": "Entry deleted successfully"}), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 3. Parameters and Return Type
The body of the request contains JSON data with the following fields:

- profile_name: The name of the player.
- drawing_name: The name of the drawing.
- score: The score achieved.

#### Example:

In [ ]:
# Example JSON payload 
{
    "profile_name": "User123",
    "drawing_name": "Drawing123",
    "score": 000
}

#Response
{
    "message": "Entry added successfully"
}

#in case of an error
{
    "error": "No data provided"
}

## 4. Call to Algorithm Request

## Definition of Code Block  
The `fetchLeaderboard` function sends a `GET` request to the API endpoint to fetch leaderboard data.  

## Call/Request to the Method  
The Fetch API is used to make the request, and the response is handled asynchronously.  

## Return/Response from the Method  
The response is parsed as JSON and passed to the `displayLeaderboard` function. Errors are logged and displayed in the DOM.  

## Handling Normal and Error Conditions  
The `submitScore` and `deleteEntry` functions handle changing data and trigger different responses based on normal and error conditions.  



#### Code:

In [ ]:
async function fetchLeaderboard() { 
        const response = await fetch(`${API_URL}/list`); #fetching the leaderboard, data handling
        if (!response.ok) throw new Error('Failed to fetch leaderboard');
        const data = await response.json(); #converting the response to json, data handling
        displayLeaderboard(data.entries || []); #display 
    } catch (error) {
        console.error('Error:', error);
        document.getElementById('leaderboard').innerHTML = 
            '<tr><td colspan="5" style="text-align: center;">Error loading leaderboard</td></tr>'; #error message
    }
}

function displayLeaderboard(entries) {  #function to display the leaderboard explicitly
    const tbody = document.getElementById('leaderboard'); #
    tbody.innerHTML = '';

    if (!entries || entries.length === 0) { 
        tbody.innerHTML = '<tr><td colspan="5" style="text-align: center;">No entries yet</td></tr>'; 
        return; 
    }

    entries.sort((a, b) => b.score - a.score)
        .forEach((entry, index) => {
            const row = document.createElement('tr');
            row.innerHTML = `
                <td>${index + 1}</td>
                <td>${entry.profile_name}</td>
                <td>${entry.drawing_name}</td>
                <td>${entry.score}</td>
                <td>
                    <button onclick="deleteEntry('${entry.profile_name}', '${entry.drawing_name}')" 
                            class="delete-btn">Delete</button>
                </td>
            `;
            tbody.appendChild(row);
        });
}

async function submitScore() { #function to submit the score, also request 
    const profileName = document.getElementById('profileName').value.trim(); 
    const drawingName = document.getElementById('drawingName').value.trim();
    const score = parseInt(document.getElementById('score').value);

    if (!profileName || !drawingName) { 
        showMessage('Please fill in all fields', true);
        return;
    }

    if (isNaN(score) || score < 0 || score > 100) {
        showMessage('Please enter a valid score between 0 and 100', true); #error message
        return;
    }

    try {
        const response = await fetch(API_URL, { 
            method: 'PUT', #updating the score
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({ #converting the data to json
                profile_name: profileName,
                drawing_name: drawingName,
                score: score
            })
        });

        const data = await response.json(); 

        if (response.ok) {
            showMessage('Score submitted successfully');
            document.getElementById('profileName').value = '';
            document.getElementById('drawingName').value = '';
            document.getElementById('score').value = '';
            await fetchLeaderboard();
        } else {
            throw new Error(data.error || 'Failed to submit score');
        }
    } catch (error) {
        console.error('Error:', error);
        showMessage(error.message, true);
    }
}

async function deleteEntry(profileName, drawingName) { #function to delete the entry, request
    if (!confirm('Are you sure you want to delete this entry?')) return;
    
    try {
        const response = await fetch(API_URL, { #fetching the data
            method: 'DELETE', #deleting the entry
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({ #converting the data to json
                profile_name: profileName,
                drawing_name: drawingName
            })
        });

        const data = await response.json();
        
        if (response.ok) {
            showMessage('Entry deleted successfully');
            await fetchLeaderboard();
        } else {
            throw new Error(data.error || 'Failed to delete entry');
        }
    } catch (error) {
        console.error('Error:', error); #error message
        showMessage(error.message, true);
    }
}